Everything to Get Current Season

Scrape

In [1]:
import pandas as pd
import csv
import os
from pathlib import Path

# Set working directory to project root if run from notebooks/
if Path.cwd().name == "notebooks":
    os.chdir("..")

# Define folders – assume they already exist
raw_data_dir = Path("data/Raw")
cleaned_data_dir = Path("data/Cleaned")
metrics_data_dir = Path("data/Metrics")
current_season_dir = metrics_data_dir / "CurrentSeasonTeams"

url = 'https://www.basketball-reference.com/wnba/years/2025_totals.html'
df2025 = pd.read_html(url)[0]  # First table is the player totals
df2025.to_csv(raw_data_dir / '2025Data.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')
df2025 = df2025[df2025['Player'] != 'Player']  # Drop header rows repeated in table
df2025 = df2025.drop_duplicates()        # Remove duplicate entries, if any
tot_players = df2025[df2025['Team'] == 'TOT']['Player']
df2025 = pd.concat([
    df2025[df2025['Team'] == 'TOT'],                              # all TOT rows
    df2025[~df2025['Player'].isin(tot_players) & (df2025['Team'] != 'TOT')]  # single-team players
], ignore_index=True)
df2025['MP'] = pd.to_numeric(df2025['MP'])
df2025 = df2025.drop('G.1', axis=1)
df2025 = df2025.drop('MP.1', axis=1)

cols_to_exclude = ['Player', 'Team', 'Pos']
cols_to_convert = df2025.columns.difference(cols_to_exclude)

df2025[cols_to_convert] = df2025[cols_to_convert].apply(pd.to_numeric, errors='coerce')

df2025

,Player,Team,Pos,G,MP,GS,FG,FGA,FG%,3P,...,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS
0,DeWanna Bonner,TOT,G-F,11,247,3,28,69,0.406,12,...,30,0.833,6,51,18,12,1,9,14,93
1,Haley Jones,TOT,F-G,8,102,0,12,26,0.462,1,...,2,0.500,3,17,7,2,3,3,12,26
2,Liatu King,TOT,F,14,136,0,13,27,0.481,1,...,15,0.733,12,29,4,2,0,11,14,38
3,Tiffany Mitchell,TOT,G,18,253,2,23,52,0.442,6,...,33,0.727,8,33,8,8,3,15,31,76
4,NaLyssa Smith,TOT,F,23,450,19,58,131,0.443,2,...,59,0.627,37,114,19,11,19,25,65,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Gabby Williams,SEA,F,22,745,22,116,257,0.451,35,...,40,0.825,15,103,99,54,11,47,34,300
164,Kiana Williams,PHO,G,4,42,0,9,15,0.600,3,...,0,NaN,0,5,4,1,0,1,0,21
165,A'ja Wilson,LVA,C,17,517,17,122,259,0.471,6,...,136,0.846,28,155,56,28,41,39,31,365
166,Jackie Young,LVA,G,21,633,21,124,284,0.437,33,...,95,0.905,27,89,81,21,8,53,55,367


Adding DRB

In [2]:
df2025['DRB'] = df2025['TRB'] - df2025['ORB']

# Define target and reference columns
col_to_move = 'DRB'
before_col = 'TRB'

# Get current column order and remove the column to move
cols = df2025.columns.tolist()
cols.remove(col_to_move)

# Find index to insert the column before the target
insert_at = cols.index(before_col)

# Insert the column at the desired position
cols.insert(insert_at, col_to_move)

# Reorder the DataFrame
df2025 = df2025[cols]

df2025.to_csv(cleaned_data_dir / '2025Data.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Metrics

In [3]:
df2025 = pd.read_csv(cleaned_data_dir / '2025Data.csv')

df2025['Efficiency'] = (1/df2025['G']) * (df2025['PTS'] + df2025['TRB'] + df2025['AST'] + df2025['STL'] - df2025['TOV'] - (df2025['FGA'] - df2025['FG']) - (df2025['FTA'] - df2025['FT']))

df2025['WinScore'] = (1/df2025['G']) * (df2025['PTS'] + df2025['TRB'] + df2025['STL'] + (df2025['AST'] * 0.5) + (df2025['BLK'] * 0.5) - df2025['FGA'] - df2025['TOV'] - (df2025['FTA'] * 0.5) - (df2025['PF'] * 0.5))

df2025['NBAPER'] = .68 + ((1/df2025['MP']) * ((26.77 * df2025['FG']) - (35.58 * df2025['FGA']) + (26.32 * df2025['FT']) - (25.47 * df2025['FTA']) + (42.74 * df2025['ORB']) + (11.45 * df2025['DRB']) + (29.7 * df2025['AST']) + (47.3 * df2025['STL']) + (38 * df2025['BLK']) - (47.9 * df2025['TOV']) - (18.8 * df2025['PF']) + (41.1 * df2025['PTS'])))

df2025['WNBAPER'] = .3289 + ((1/df2025['MP']) * ((53.35 * df2025['FG']) - (29.28 * df2025['FGA']) + (15.27 * df2025['FT']) - (25.34 * df2025['FTA']) + (47.72 * df2025['ORB']) + (10.95 * df2025['DRB']) + (31.61 * df2025['AST']) + (50.72 * df2025['STL']) + (50.68 * df2025['BLK']) - (45.88 * df2025['TOV']) - (61.23 * df2025['PF']) + (41.1 * df2025['PTS'])))

print("Columns before filtering (2025):", df2025.columns.tolist())

expected_cols = ['Player', 'Team', 'Pos', 'WinScore', 'Efficiency', 'NBAPER', 'WNBAPER']

df2025 = df2025[[col for col in expected_cols if col in df2025.columns]]

df2025.to_csv(current_season_dir / '2025Metrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

df2025teamless = df2025[['Player', 'Pos', 'WinScore', 'Efficiency', 'NBAPER', 'WNBAPER']].copy()

df2025teamless = df2025teamless.astype({
    "Player": str, "Pos": str, 
    "WinScore": float, "Efficiency": float, "NBAPER": float, "WNBAPER": float
})

df2025teamless = df2025teamless.round(3)

df2025teamless.to_csv(metrics_data_dir / '2025Metrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')

Columns before filtering (2025): ['Player', 'Team', 'Pos', 'G', 'MP', 'GS', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Efficiency', 'WinScore', 'NBAPER', 'WNBAPER']


Team Specific

In [4]:
df2025 = pd.read_csv(current_season_dir / '2025Metrics.csv')

df2025 = df2025.round(3)

# The column to split on
split_column = "Team"

# Directory to save files (ensure it exists or create it)
output_dir = current_season_dir

# Split and save
for group, group_df in df2025.groupby(split_column):
    filename = f"{output_dir}/{group}.csv"
    group_df = group_df.drop(columns=['Team'])  # Drop 'Team' column
    group_df.to_csv(filename, index=False, sep=',', encoding='utf-8', lineterminator='\n')

Combined DFs

In [5]:

df2023 = pd.read_csv(metrics_data_dir / '2023Metrics.csv')

df2024 = pd.read_csv(metrics_data_dir / '2024Metrics.csv')

df2025 = pd.read_csv(metrics_data_dir / '2025Metrics.csv')

combined = pd.concat([
    df2023.assign(Season=2023),
    df2024.assign(Season=2024),
    df2025.assign(Season=2025)
])

combined[['Player', 'Season', 'Pos', 'WinScore', 'Efficiency', 'NBAPER', 'WNBAPER']].to_csv(metrics_data_dir / 'AllPlayersMetrics.csv', index=False, sep=',', encoding='utf-8', lineterminator='\n')